# FileBasedCache - bi-directional storage primitives for Apache Beam

## Install dependencies

In [0]:
!sudo apt-get -yqq install libsnappy-dev

In [0]:
!pip install -q python-snappy Faker

In [0]:
!pip install "git+https://github.com/ostrokach/beam.git@feature/filebasedcache#egg=apache_beam[gcp]&subdirectory=sdks/python"

## Imports

In [0]:
import copy
import itertools
import logging
import os
import tempfile
import uuid
import pickle

import apache_beam as beam
import fastavro
import numpy as np
import pandas as pd
import tqdm
from apache_beam.io.filesystems import FileSystems
from apache_beam.options.pipeline_options import (GoogleCloudOptions,
                                                  PipelineOptions)
from apache_beam.runners.direct.direct_runner import BundleBasedDirectRunner
from apache_beam.runners.interactive import caching
from apache_beam.transforms.ptransform import ptransform_fn
from faker import Faker

## Parameters

In [0]:
temp_location = tempfile.mkdtemp(prefix="beam-dev-")

cache_location = tempfile.mkdtemp(dir=temp_location, prefix="cache-")

temp_location, cache_location

('/tmp/beam-dev-ZxP49F', '/tmp/beam-dev-ZxP49F/cache-35ocoU')

In [0]:
options = PipelineOptions(runner="direct", temp_location=temp_location)
options.display_data()

{'runner': 'direct', 'temp_location': '/tmp/beam-dev-ZxP49F'}

## Load data

### Generate a dataset of fake people profiles

In [0]:
try:
    with open("fake_people.pkl", "rb") as f:
        fake_people = pickle.load(f)
except IOError:
    fake = Faker(42)
    fake_people = [fake.profile(fields=None, sex=None) for _ in tqdm.tqdm_notebook(range(10000))]
    with open("fake_people.pkl", "wb") as f:
        pickle.dump(fake_people,f )

In [0]:
fake_people[0]

{'address': u'399 William Groves\nMorganhaven, WA 49272',
 'birthdate': datetime.date(1962, 4, 24),
 'blood_group': 'B+',
 'company': u'Thomas, Cruz and Small',
 'current_location': (Decimal('-62.4194185'), Decimal('131.920989')),
 'job': 'Ecologist',
 'mail': u'goodmananna@gmail.com',
 'name': u'Steven Watkins',
 'residence': u'394 Rachel Mews\nBeardmouth, AR 30354',
 'sex': 'M',
 'ssn': u'475-46-1888',
 'username': u'smiller',
 'website': [u'http://schultz.net/']}

## Analyse data

### Create a PCollection cache

Convert the generated dataset into a `PCollection` which can be accessed from within a Beam pipeline.

In [0]:
input_cache = caching.SafeTextBasedCache(FileSystems.join(cache_location, "fake_people"), if_exists="overwrite")
input_cache.write(fake_people)

In [0]:
next(input_cache.read())

{'address': u'399 William Groves\nMorganhaven, WA 49272',
 'birthdate': datetime.date(1962, 4, 24),
 'blood_group': 'B+',
 'company': u'Thomas, Cruz and Small',
 'current_location': (Decimal('-62.4194185'), Decimal('131.920989')),
 'job': 'Ecologist',
 'mail': u'goodmananna@gmail.com',
 'name': u'Steven Watkins',
 'residence': u'394 Rachel Mews\nBeardmouth, AR 30354',
 'sex': 'M',
 'ssn': u'475-46-1888',
 'username': u'smiller',
 'website': [u'http://schultz.net/']}

### Validate the PCollection cache

In [0]:
# Make sure we have 10_000 fake profiles, as expected
temp = caching.TFRecordBasedCache(
    FileSystems.join(cache_location, "temp"), if_exists="overwrite"
)

with beam.Pipeline(options=options) as p:
    _ = (
        p
        | input_cache.reader()
        | beam.combiners.Count.Globally()
        | temp.writer()
    )

assert next(temp.read()) == 10000

### Select people with duplicate usernames

In [0]:
# Make sure every username is distinct
temp = caching.TFRecordBasedCache(
    FileSystems.join(cache_location, "temp"), if_exists="overwrite"
)

with beam.Pipeline(options=options) as p:
    _ = (
        p
        | input_cache.reader()
        | "Extract username" >> beam.WithKeys(lambda e: e["username"])
        | "Group people by username" >> beam.GroupByKey(lambda e: e)
        | beam.Values()
        | "Select groups with at least two people" >> beam.Filter(lambda vs: len(vs) >= 2)
        | "Ungroup" >> beam.FlatMap(lambda gp: [e for e in gp])
        | temp.writer()
    )

In [0]:
pd.DataFrame(temp.read()).sort_values("username")

,address,birthdate,blood_group,company,current_location,job,mail,name,residence,sex,ssn,username,website
450,6141 Gibbs Gateway Suite 312\nNorth Jessicavie...,2003-04-12,AB-,Smith-Washington,"(-65.287492, -163.373159)","Education officer, museum",claytonallen@hotmail.com,Misty Santiago,"31819 Leah Gardens Apt. 190\nPort Anthonyland,...",F,007-91-5069,aaron60,"[http://www.pace-jordan.com/, https://cervante..."
449,USNV Austin\nFPO AE 27822,1993-05-26,O+,Robertson-Rivera,"(-40.7268285, -163.738816)","Administrator, education",kaitlyn73@hotmail.com,Michael Todd,"8560 Pamela Crossing\nLake Carolyn, IN 33635",M,824-04-5069,aaron60,"[https://www.salazar.net/, http://www.herrera...."
873,"46860 Henderson Ports\nWest Meganmouth, IL 14690",1911-04-30,O-,Jones and Sons,"(41.9038925, 16.415893)",Chief Operating Officer,erika50@gmail.com,Thomas Cummings,Unit 0947 Box 1633\nDPO AE 47811,M,754-91-7079,abarnes,[https://www.shelton.biz/]
872,"994 Watson Wells\nDominguezmouth, MD 10842",1995-04-05,A-,Alvarez-Shah,"(76.333256, -48.021819)","Physicist, medical",whitneysherri@gmail.com,Adrienne Gonzales,"34277 Devin Locks Apt. 508\nEast Blakeport, TN...",F,409-69-0709,abarnes,"[https://www.goodwin-gonzalez.com/, http://joh..."
651,"41789 Dawson Lights Apt. 198\nOwensberg, OR 21880",1963-03-01,B+,"Arias, Smith and Johnson","(-9.570993, -19.123235)",Hospital doctor,dsmith@hotmail.com,Jamie Rich,"4040 Anthony Harbors\nDanielchester, OR 78117",F,593-96-3443,abrooks,"[http://parker.com/, https://www.torres-rivera..."
650,"7319 Julie Mission\nGarciaberg, NV 25150",1988-08-04,B-,Pena Group,"(-88.7618415, -125.202564)","Engineer, maintenance (IT)",colinthompson@gmail.com,Kyle Harris,Unit 5075 Box 6101\nDPO AE 42941,M,856-52-0807,abrooks,[https://www.kaiser.com/]
119,"30352 Jonathan Land\nHernandeztown, CT 55694",2016-07-24,O+,Mccarthy Group,"(-30.0746895, -91.522030)",Oceanographer,skim@yahoo.com,Monica Rowland,"86903 Joshua Lake Apt. 471\nWilliamsview, AK 5...",F,644-87-0386,acarter,"[https://hunt.com/, https://klein.net/, https:..."
118,"86395 William Vista\nKaitlynmouth, NJ 83811",2012-05-23,B-,"Scott, Solis and Singh","(22.1210015, 12.968199)",Landscape architect,alan10@hotmail.com,Gerald Faulkner,"1342 Reginald Prairie\nNorth Jasmine, MS 88404",M,546-44-6054,acarter,"[http://www.griffin.com/, http://smith.org/, h..."
35,"8058 Parker Orchard\nDanielmouth, WV 58356",1912-06-11,B-,"Murphy, Hodge and Li","(32.842872, -53.791196)",Land,tateleslie@hotmail.com,Kimberly Petersen,"40586 Stewart Plaza\nNorth Jennifer, WI 93728",F,714-01-1485,adamsdavid,"[https://huffman-hodge.com/, https://www.allen..."
36,"789 Smith Hollow Suite 406\nMillerland, NE 07759",1915-01-06,O+,"Wilcox, Moreno and Miller","(-75.712635, -175.328974)",Cartographer,choibrianna@gmail.com,Sabrina Hanson,"640 Bass Parks\nPort Robinmouth, DE 93315",F,724-97-7324,adamsdavid,"[https://www.whitney-wang.net/, https://www.jo..."


### Find most popular occupations

Select top 10 most popular occupations in the `> 30` age group.

In [0]:
def calculate_age(birthdate):
    import datetime
    td = datetime.date.today() - birthdate
    return int(round(td.days // 365.25))

def add_age_colum(element):
    element["age"] = calculate_age(element["birthdate"])
    return element

In [0]:
most_popular_occupations = caching.TFRecordBasedCache(
    FileSystems.join(cache_location, "most_popular_occupations"), if_exists="overwrite"
)

with beam.Pipeline(options=options) as p:
    _ = (
        p
        | input_cache.reader()
        | beam.Map(add_age_colum)
        | beam.Filter(lambda e: e["age"] > 30)
        | beam.Map(lambda e: (e["job"], e))
        | beam.combiners.Count.PerKey()
        | beam.combiners.Top.Of(10, key=lambda x: x[1])
        | most_popular_occupations.writer()
    )

In [0]:
next(most_popular_occupations.read())

[('Management consultant', 27),
 ('Personal assistant', 22),
 ('Music tutor', 21),
 ('Warden/ranger', 21),
 ('Homeopath', 21),
 ('Fast food restaurant manager', 20),
 ('Art gallery manager', 20),
 ('Games developer', 20),
 ('Production engineer', 19),
 ('Product/process development scientist', 19)]

## Process and export data

In [0]:
def normalize_data(element):
    return {
        "username": element["username"],
        "first_name": element["name"].split()[0],
        "last_name": " ".join(element["name"].split()[1:]),
        "mail": element["name"],
        "job": element["name"],
        "company": element["name"],
        "age": calculate_age(element["birthdate"]),
        "current_location_lat": float(element["current_location"][0]),
        "current_location_long": float(element["current_location"][1]),
        "website": element["website"] or [],
    }

normalize_data(element)

{'age': 64,
 'company': u'Gary Lewis',
 'current_location_lat': 28.4744555,
 'current_location_long': -77.483937,
 'first_name': u'Gary',
 'job': u'Gary Lewis',
 'last_name': u'Lewis',
 'mail': u'Gary Lewis',
 'username': u'raymondparker',
 'website': [u'http://www.rivera.info/',
  u'http://kelly-mcdowell.com/',
  u'https://white-yu.com/',
  u'http://brown.biz/']}

In [0]:
avro_schema = fastavro.parse_schema({
    "namespace": "example.avro",
    "name": "User",
    "type": "record",
    "fields": [
        {"name": "username", "type": "string"},
        {"name": "first_name", "type": "string"},
        {"name": "last_name", "type": "string"},
        {"name": "mail", "type": "string"},
        {"name": "job", "type": "string"},
        {"name": "company", "type": "string"},
        {"name": "age", "type": "int"},
        {"name": "current_location_lat", "type": "double"},
        {"name": "current_location_long", "type": "double"},
        {"name": "website", "type": {"type": "array", "items": "string"}},
    ],
})
avro_schema

{'__fastavro_parsed': True,
 'fields': [{'name': 'username', 'type': 'string'},
  {'name': 'first_name', 'type': 'string'},
  {'name': 'last_name', 'type': 'string'},
  {'name': 'mail', 'type': 'string'},
  {'name': 'job', 'type': 'string'},
  {'name': 'company', 'type': 'string'},
  {'name': 'age', 'type': 'int'},
  {'name': 'current_location_lat', 'type': 'double'},
  {'name': 'current_location_long', 'type': 'double'},
  {'name': 'website', 'type': {'items': 'string', 'type': 'array'}}],
 'name': 'example.avro.User',
 'type': 'record'}

In [0]:
output = caching.AvroBasedCache(
    FileSystems.join(cache_location, "temp"), if_exists="overwrite", schema=avro_schema,
    use_fastavro=True,
)

with beam.Pipeline(options=options) as p:
    _ = (
        p
        | input_cache.reader()
        | beam.Map(normalize_data)
        | output.writer()
    )

In [0]:
next(output.read())

{u'age': 57,
 u'company': u'Steven Watkins',
 u'current_location_lat': -62.4194185,
 u'current_location_long': 131.920989,
 u'first_name': u'Steven',
 u'job': u'Steven Watkins',
 u'last_name': u'Watkins',
 u'mail': u'Steven Watkins',
 u'username': u'smiller',
 u'website': [u'http://schultz.net/']}

## Write SQL queries against the result

### Google Cloud

## Export data to Google Cloud

In [0]:
#@title Google Cloud Project Info { display-mode: "form" }
project_id = "asdfdsf" #@param {type:"string"}
gcs_temp_location = "" #@param {type:"string"}

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
gcs_output = caching.AvroBasedCache(
    FileSystems.join(gcs_temp_location, "filebasedcache-output"), if_exists="overwrite",
    schema=avro_schema, use_fastavro=True)

gcs_output.write(output.read())

In [0]:
next(gcs_output.read())

IOError: ignored

### Query data using BigQuery

In [0]:
from google.cloud import bigquery

In [0]:
options = PipelineOptions(runner="direct", temp_location=gc_temp_location, staging_location=gsc_temp_location, project=project_id)
options.display_data()

{'project': 'strokach-playground',
 'runner': 'direct',
 'staging_location': 'gs://strokach/temp',
 'temp_location': 'gs://strokach/temp'}

In [0]:
FileSystems.mkdirs(gsc_temp_location)
FileSystems.mkdirs(FileSystems.join(gsc_temp_location, "filebasedcache-demo"))

In [0]:
gcs_output = caching.AvroBasedCache(
    FileSystems.join(gcs_temp_location), if_exists="overwrite",
    schema=avro_schema, use_fastavro=True)

# gce_output.write(output.read())


with beam.Pipeline(options=options) as p:
    _ = (
        p
        | input_cache.reader()
        | beam.Map(normalize_data)
        | output.writer()
    )

BeamIOError: ignored

In [0]:
gcs_output._existing_file_paths()

[u'gs://strokach/temp/filebasedcache-output-001']

In [0]:
# Configure the external data source and query job
table_id = 'fake_people'
external_config = bigquery.ExternalConfig('AVRO')
external_config.source_uris = gcs_output._existing_file_paths()

# external_config.schema = [
#     bigquery.SchemaField('name', 'STRING'),
#     bigquery.SchemaField('post_abbr', 'STRING')
# ]
# external_config.options.skip_leading_rows = 1  # optionally skip header row

In [0]:
external_config.to_api_repr()

{'sourceFormat': 'AVRO',
 'sourceUris': [u'gs://strokach/temp/filebasedcache-output-001']}

In [0]:
client = bigquery.Client(project=project_id)

In [0]:
# Example query to find states starting with 'W'
sql_query = """
SELECT *
FROM `{table_id}`
WHERE AGE = (
    SELECT MAX(age)
    FROM `{table_id}`
)
""".format(table_id=table_id)

job_config = bigquery.QueryJobConfig()
job_config.table_definitions = {table_id: external_config}

query_job = client.query(sql_query, job_config=job_config)

In [0]:
query_job.to_dataframe()

,username,first_name,last_name,mail,job,company,age,current_location_lat,current_location_long,website
0,georgemorrison,Colleen,Reynolds,Colleen Reynolds,Colleen Reynolds,Colleen Reynolds,115,-79.006885,-75.563425,"[https://wilson-casey.info/, https://palmer.com/]"
1,smithchristine,Rebecca,Walker,Rebecca Walker,Rebecca Walker,Rebecca Walker,115,-81.249215,72.618745,"[http://mcknight-lawrence.net/, https://fitzge..."
2,martinlaura,Margaret,Peters,Margaret Peters,Margaret Peters,Margaret Peters,115,63.213485,-174.161011,"[http://www.robinson-walker.com/, http://www.b..."
3,walterslisa,Bernard,Krause,Bernard Krause,Bernard Krause,Bernard Krause,115,74.522577,117.395325,"[http://www.stout.org/, http://burke-jones.com/]"
4,alexanderoneill,Mrs.,Maureen Williams,Mrs. Maureen Williams,Mrs. Maureen Williams,Mrs. Maureen Williams,115,28.013755,-142.637446,"[http://wolf.info/, https://reed.net/, https:/..."
5,palmerdana,Tony,Contreras,Tony Contreras,Tony Contreras,Tony Contreras,115,69.300890,-80.520800,"[https://www.brooks.com/, http://spence.com/]"
6,christopher87,William,Graves,William Graves,William Graves,William Graves,115,11.095429,177.999067,"[https://mcpherson-brown.info/, http://www.col..."
7,angelaroth,Vicki,Sanchez,Vicki Sanchez,Vicki Sanchez,Vicki Sanchez,115,61.692383,-15.751065,"[https://cunningham.biz/, http://moore.com/, h..."
8,jamesriley,Noah,Ramirez,Noah Ramirez,Noah Ramirez,Noah Ramirez,115,-15.383256,96.422860,"[http://www.torres.com/, http://www.harris.com..."
9,alexandra66,Mary,Flowers,Mary Flowers,Mary Flowers,Mary Flowers,115,83.523985,-141.297647,"[http://smith.org/, https://guerra.com/, https..."


## Cleanup

In [0]:
shutil.rmtree(temp_location)